In [1]:
import os
import numpy
from ctypes import *
# from scipy import misc
import time
from pyAndorSDK3 import AndorSDK3
import matplotlib.pyplot as plt
from tqdm import tqdm



In [2]:

def plot_intensity(images, slm_array):
    # average the intensity of the cam
    intensity = numpy.mean(images, axis=(1,2))
    max_intensity = numpy.max(intensity)
    min_intensity = numpy.min(intensity)
    uncentered_intensity = intensity/max_intensity
    intensity = (intensity-min_intensity)/(max_intensity-min_intensity)
    print(f'shape: {slm_array[:,0,0].shape}, {slm_array[0,:,0].shape}, {slm_array[0,0,:].shape}')
    # display pixel value of SLM vs. image value cam (-> intensity)
    fig, (ax1, ax2) = plt.subplots(2, 1)
    indices = numpy.linspace(0, 29, num=30)
    ax1.plot(slm_array[:,0,0], intensity, '.-')
    #ax1.plot(indices, intensity, '.-')
    ax1.set_ylabel('intensity')
    ax1.set_xlabel('grayscale value')

    # plot phase vs. pixel value
    phase = numpy.arccos(2*intensity - 1)/(numpy.pi)
    ax2.plot(slm_array[:,0,0], phase, '.-')
    #ax2.plot(indices, phase, '.-')
    ax2.set_ylabel('phase')
    ax2.set_xlabel('grayscale value')
    plt.savefig(".\calibration_1_curr.png")
    plt.show()
    plt.close()

def AOI_cam(cam, binning='1x1', width=2048, height=2048, left=1, top=1, exp_time=0.001, cycle_mode='Continuous'):
    cam.AOIBinning = binning
    cam.AOIWidth = width
    cam.AOILeft = left
    cam.AOIHeight = height
    cam.AOITop = top
    # cam.framerate=90
    cam.ExposureTime = exp_time
    cam.CycleMode = cycle_mode

def temperature_stabilisation(cam):
    # Cooling the camera sensor
    cam.SensorCooling = True
    # waiting for temperature to stabilise
    while(cam.TemperatureStatus != "Stabilised"):
        time.sleep(5)
        print("Temperature: {:.5f}C".format(cam.SensorTemperature), end="  ")
        print("Status: '{}'".format(cam.TemperatureStatus))
        if cam.TemperatureStatus == "Fault":
            err_str = "Camera faulted when cooling to target temperature"
            raise RuntimeError(err_str)

    print("Sensor Temperature now Stabilised and Camera is ready to use")



In [ ]:

# Query DPI Awareness (Windows 10 and 8)
import ctypes
awareness = ctypes.c_int()
errorCode = ctypes.windll.shcore.GetProcessDpiAwareness(0, ctypes.byref(awareness))
print(awareness.value)

# Set DPI Awareness  (Windows 10 and 8)
errorCode = ctypes.windll.shcore.SetProcessDpiAwareness(2)

# Set DPI Awareness  (Windows 7 and Vista)
success = ctypes.windll.user32.SetProcessDPIAware()

# Load DLL for SLM functionalities
cdll.LoadLibrary("C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\SDK\\Blink_C_wrapper")
slm_lib = CDLL("Blink_C_wrapper") # slm_lib.function() can now be uesd like the functions in C but in python

# Load DLL for image generation
cdll.LoadLibrary("C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\SDK\\ImageGen")
image_lib = CDLL("ImageGen")


# Call the constructor for the SLM
slm_lib.Create_SDK(1)

# load LUT
file_path = "C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI\\LUT Files\\custom_linear_LUT.LUT"
file_path_c = ctypes.c_char_p(file_path.encode('utf-8'))
slm_lib.Load_lut(file_path_c)

# indicate that our images are black and white -> don't really see why this is necessary?
RGB = c_uint(0)
is_eight_bit_image = c_uint(1)

# SLM dimensions
height = c_uint(slm_lib.Get_Height())
width = c_uint(slm_lib.Get_Width())
depth = c_uint(slm_lib.Get_Depth())
bytesPerPixel = 1 #for not RGBA? #4; #RGBA
center_x = c_uint(width.value//2)
center_y = c_uint(height.value//2)
print("height: ", height.value, "width: ", width.value, "depth: ", depth.value)


0


In [5]:
# connect to camera and set parameters
sdk3 = AndorSDK3()
cam = sdk3.GetCamera(0)
AOI_cam(cam,
        width=400,
        height=400,
        left=835,
        top=765,
)  

# Cooling the camera sensor
cam.SensorCooling = True
temperature_stabilisation(cam)



CameraException: Error Opening Camera Index 0 - 38 (AT_ERR_DEVICEINUSE)

In [9]:
# generate the 256 patterns
sample_num=256
pattern = numpy.ones((height.value, width.value), dtype=numpy.uint8) 
slm_array = numpy.empty([sample_num, height.value, width.value], dtype=numpy.uint8)
for i in range(sample_num):
    slm_array[i] = pattern * i
    print(f'pattern: {slm_array[i]}')


pattern: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
pattern: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
pattern: [[2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 ...
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]]
pattern: [[3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 ...
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]]
pattern: [[4 4 4 ... 4 4 4]
 [4 4 4 ... 4 4 4]
 [4 4 4 ... 4 4 4]
 ...
 [4 4 4 ... 4 4 4]
 [4 4 4 ... 4 4 4]
 [4 4 4 ... 4 4 4]]
pattern: [[5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]
 ...
 [5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]]
pattern: [[6 6 6 ... 6 6 6]
 [6 6 6 ... 6 6 6]
 [6 6 6 ... 6 6 6]
 ...
 [6 6 6 ... 6 6 6]
 [6 6 6 ... 6 6 6]
 [6 6 6 ... 6 6 6]]
pattern: [[7 7 7 ... 7 7 7]
 [7 7 7 ... 7 7 7]
 [7 7 7 ... 7 7 7]
 ...
 [7 7 7 ... 7 7 7]
 [7 7 7

In [ ]:


# loop for measurements with cam!
images = numpy.empty([sample_num, cam.AOIWidth, cam.AOIHeight], numpy.uint8, 'C')
pbar = tqdm(range(sample_num))
for k in pbar:
    slm_array_ptr = slm_array[k].flatten().ctypes.data_as(POINTER(c_ubyte))
    # define WFC as zero (what alfonso did)
    wfc = pattern * 0
    wfc_ptr = wfc.flatten().ctypes.data_as(POINTER(c_ubyte))
    # generate the images for the SLM
    image_lib.Generate_Solid(slm_array_ptr, wfc_ptr, width.value, height.value, depth.value, is_eight_bit_image, RGB)
    # write the image on the slm
    slm_lib.Write_image(slm_array_ptr, is_eight_bit_image)
    if k == 0:
        time.sleep(1) # Alfonso did this for the first image because the SLM needs a bit longer for the intial big jump
    time.sleep(0.04)
    # measure the intensity
    img = cam.acquire(timeout=50, min_buf=4)
    images[k] = img.image
    if k % 5 == 0:
        #img.save(f'.\cam_img_{k}.png')
        plt.imshow(images[k], cmap='gray', vmin=0, vmax=255)
        plt.savefig(f'.\cam_img_step{k}.png')

    pbar.set_postfix({'Temperature': cam.SensorTemperature})

plot_intensity(images=images, slm_array=slm_array)

# Always call Delete_SDK before exiting
slm_lib.Delete_SDK()


In [21]:
stripes = pattern
stripe_color = 0
period = 2
for i in range(1920):
    if i%period == 0:
        stripe_color = ((i/period)%2==0)*86
    stripes[:,i]=stripe_color
print(stripes)
        

slm_lib.Write_image(stripes.flatten().ctypes.data_as(POINTER(c_ubyte)), is_eight_bit_image)

[[86 86  0 ... 86  0  0]
 [86 86  0 ... 86  0  0]
 [86 86  0 ... 86  0  0]
 ...
 [86 86  0 ... 86  0  0]
 [86 86  0 ... 86  0  0]
 [86 86  0 ... 86  0  0]]


0

In [5]:


# loop for measurements no cam!
pbar = tqdm(range(sample_num))
for k in pbar:
    slm_array_ptr = slm_array[k].flatten().ctypes.data_as(POINTER(c_ubyte))
    # define WFC as zero (what alfonso did)
    wfc = pattern * 0
    wfc_ptr = wfc.flatten().ctypes.data_as(POINTER(c_ubyte))
    # generate the images for the SLM
    image_lib.Generate_Solid(slm_array_ptr, wfc_ptr, width.value, height.value, depth.value, is_eight_bit_image, RGB)
    # write the image on the slm
    slm_lib.Write_image(slm_array_ptr, is_eight_bit_image)
    if k == 0:
        time.sleep(1) # Alfonso did this for the first image because the SLM needs a bit longer for the intial big jump
    time.sleep(0.1)

# Always call Delete_SDK before exiting
slm_lib.Delete_SDK()


  0%|          | 0/256 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:38<00:00,  6.69it/s]


-2105513104

In [21]:
import numpy as np
import ctypes
import time

# Assuming the C library Blink_C_wrapper has been loaded
# For example, it could be something like:
# slm_lib = ctypes.CDLL('path/to/Blink_C_wrapper.so')  # On Linux/Mac
# slm_lib = ctypes.CDLL('path\\to\\Blink_C_wrapper.dll')  # On Windows

# Example SLM class (you may already have this defined)
class SLM:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def ImgWrite(self, img):
        # Convert image to uint8
        img = np.uint8(img * 255)
        
        # Reshape the image to match the expected input shape (1 x width*height)
        reshaped_img = np.reshape(img.T, (1, self.width * self.height))
        
        # Assuming Write_image is a C function in the Blink_C_wrapper library
        # We need to set up the Write_image function to expect a ctypes array
        img_ptr = reshaped_img.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte))
        
        # Call the C function Write_image
        slm_lib.Write_image(img_ptr, 1)
        
        # Small pause (similar to MATLAB's pause)
        time.sleep(0.1)

# Function to run the loop
def run_slm_image_loop(slm):
    for i in range(256):  # Loop from 0 to 255
        k = i / 256  # Calculate the intensity value (same as MATLAB loop)
        
        # Create a uniform image of size (1920, 1200)
        slm_image = k * np.ones((1920, 1200))
        
        # Display the image (equivalent to MATLAB's imagesc)
        import matplotlib.pyplot as plt
        
        # Write the image to the SLM
        slm.ImgWrite(slm_image)
        
        # Pause (equivalent to MATLAB's pause)
        time.sleep(0.01)

# Initialize the SLM with dimensions
slm = SLM(1920, 1200)

# Run the loop to write images to the SLM
run_slm_image_loop(slm)